In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import Latex

from src.test_ import FTest, TTest

$ H_0: b_1 = ... = b_k = 0 $
$ H_1: b_1^2 + ... + b_k^2 > 0 $
$$ H_0: b_1 = ... = b_k = 0, \\ H_1: b_1^2 + ... + b_k^2 > 0. $$

$ H_0: b_i = 0 $
$ H_1: b_i \neq 0 $
$$ H_0: b_i = 0, \\ H_1: b_i \neq 0. $$


In [12]:
Latex(FTest.hypotheses(details=1))

<IPython.core.display.Latex object>

In [13]:
Latex(TTest.hypotheses())

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>